In [274]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

## **1. Akses Dataset**

Akses dataset serangan hama dengan 7 jenis hama berbeda yang disertai kategori :

(T) -> jumlah area yang terkena serangan

(P) -> jumlah area yang mengalami kerusakan lebih dari 75%

In [275]:
dataset_dir = "C:\\Users\\Diputra_W\\Documents\\Campus\\Study\\Mata Kuliah\\Semester 7\\PKL\\FInal Project\\src\\penerapan-pengelolaan-hama-terpadu-tanaman-pangan.xlsx"

In [276]:
df = pd.ExcelFile(dataset_dir)

# Akses tiap halaman pada dataset (jika ada)
df.sheet_names

['padi',
 'jagung',
 'kedelai',
 'kacang tanah',
 'kacang hijau',
 'ubi kayu',
 'ubi jalar']

In [277]:
# Load data untuk halaman tertentu & tampilkan
padi_data = df.parse('padi')

print(f"Shape data : {padi_data}")
padi_data.head(178)

Shape data :           TAHUN  NO PROV            PROV PENGGEREK BATANG PADI Unnamed: 4   
0           NaN      NaN             NaN                     T          P  \
1          2018      1.0            Aceh                2818.7          0   
2          2018      2.0  Sumatera Utara                1799.0          0   
3          2018      3.0  Sumatera Barat                116.15          0   
4          2018      4.0            Riau                 952.9          0   
..          ...      ...             ...                   ...        ...   
171        2022     31.0          Maluku               1422.75          0   
172        2022     32.0    Maluku Utara                 282.8          0   
173        2022     33.0     Papua Barat                 645.7          0   
174        2022     34.0           Papua                 259.4          0   
175  2022 Total      NaN             NaN             87199.623     150.44   

              WBC  Unnamed: 6         TIKUS Unnamed: 8       B

,TAHUN,NO PROV,PROV,PENGGEREK BATANG PADI,Unnamed: 4,WBC,Unnamed: 6,TIKUS,Unnamed: 8,BLAS,Unnamed: 10,KRESEK,Unnamed: 12,TUNGRO,Unnamed: 14,KR/KH,Unnamed: 16,TOTAL OPUT,Unnamed: 18
0,NaN,NaN,NaN,T,P,T,P,T,P,T,P,T,P,T,P,T,P,T,P
1,2018,1.0,Aceh,2818.7,0,2663.48,95.25,2722.45,0,1409,0,2738,0,0,0,0,0,12351.63,95.25
2,2018,2.0,Sumatera Utara,1799.0,0,478.56,3.18,2030.15,22.5,3042.95,0,1831.5,0,35.1,0,68.4,0,9285.66,25.68
3,2018,3.0,Sumatera Barat,116.15,0,430.7,11.6,2044.21,136.25,465.01,14.05,31.25,0.5,104.85,5.25,57.45,14.5,3249.62,182.15
4,2018,4.0,Riau,952.9,0,435.35,0,658.25,1,485.65,0,117.55,0,25,0,NaN,NaN,2674.7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,2022,31.0,Maluku,1422.75,0,203.5,2.5,295,0,48,0,26.75,0,5.5,0,NaN,NaN,2001.5,2.5
172,2022,32.0,Maluku Utara,282.8,0,9,0,151.5,0,8.5,0,0,0,13.4,0,NaN,NaN,465.2,0
173,2022,33.0,Papua Barat,645.7,0,229.8,0,34.4,0,48.5,0,61,0,NaN,NaN,NaN,NaN,1019.4,0
174,2022,34.0,Papua,259.4,0,9,0,18.25,0,333.9,0,626.5,0,1179.25,0,NaN,NaN,2426.3,0


## **2. Preprocessing Data**

Membersihkan dataset dan mempersiapkan untuk dilakukan regresi linear :

- Perbaiki header

- Isi kolom -, null, nan, dengan nilai 0

- Pilih kolom/baris yang ingin digunakan dalam sistem

In [278]:
# Perbaikan header data
data = pd.read_excel(dataset_dir, header=[0, 1])
data.columns = [" ".join(col).strip() for col in data.columns.values]

# Cleaning kolom data
data.columns = data.columns.str.strip()
data.columns = data.columns.str.replace(" ", "_")
data.columns = data.columns.str.replace("/", "_")
data.columns = data.columns.str.replace("-", "_")

# Ganti nama kolom (tujuan untuk mempermudah proses kedepannya)
data.rename(columns={
    "TAHUN_Unnamed:_0_level_1": "TAHUN",
    "NO_PROV_Unnamed:_1_level_1": "KODE_PROVINSI",
    "PROV_Unnamed:_2_level_1": "PROVINSI",
}, inplace=True)

# Penanganan missing values
data.fillna(0, inplace=True)

# Menghilangkan baris yang tidak diperlukan
rows_to_drop = [34, 69, 104, 139, 174]
data = data.drop(index=rows_to_drop)

# Filtering kolom yang akan digunakan (kode provinsi, tahun, dan kolom _T)
columns_to_keep = ['KODE_PROVINSI'] + [col for col in data.columns if '_T' in col]
data_T = data[columns_to_keep]

data_T.head(40)

,KODE_PROVINSI,PENGGEREK_BATANG_PADI_T,WBC_T,TIKUS_T,BLAS_T,KRESEK_T,TUNGRO_T,KR_KH_T,TOTAL_OPUT_T
0,1.0,2818.700,2663.4800,2722.450,1409.000,2738.00,0.000,0.00,12351.6300
1,2.0,1799.000,478.5600,2030.150,3042.950,1831.50,35.100,68.40,9285.6600
2,3.0,116.150,430.7000,2044.210,465.010,31.25,104.850,57.45,3249.6200
3,4.0,952.900,435.3500,658.250,485.650,117.55,25.000,0.00,2674.7000
4,5.0,345.980,70.4000,448.360,169.230,61.49,4.410,0.00,1099.8700
5,6.0,4681.480,1728.5700,8608.060,3076.300,2313.05,83.270,135.90,20626.6300
6,7.0,837.950,60.2500,796.240,614.100,285.75,34.500,0.50,2629.2900
7,8.0,4717.000,1842.0000,6765.000,3071.000,2885.00,1.000,26.00,19307.0000
8,9.0,17.420,424.1000,214.950,175.700,40.43,0.000,0.00,872.6000
9,10.0,0.000,0.0000,0.000,0.000,0.00,0.000,0.00,0.0000


Sesuai catatan sebelumnya, model akan mengguakan data kategori (T) dan dijumlahkan total untuk 1 provinsi untuk setiap tahunnya.

In [279]:
# # Jumlahkan data dalam 5 tahun
# data_T = data_T.groupby('KODE_PROVINSI').sum().reset_index()

# #print(f"Shape Data Final : {data_T.shape}")
# data_T.head(40)

## **3. Filtering Data**

Filtering bagian data yang akan digunakan dalam model regresi linear. Tentukan mana variabel bebas dan terikat.

In [280]:
# Variabel dependen
Y_T = data_T["TOTAL_OPUT_T"]

X_T = [col for col in data_T.columns if '_T' in col and col != "TOTAL_OPUT_T"]
X_T = data_T[X_T]

In [281]:
print(X_T)
print(X_T.shape)

     PENGGEREK_BATANG_PADI_T    WBC_T  TIKUS_T   BLAS_T  KRESEK_T  TUNGRO_T   
0                    2818.70  2663.48  2722.45  1409.00   2738.00      0.00  \
1                    1799.00   478.56  2030.15  3042.95   1831.50     35.10   
2                     116.15   430.70  2044.21   465.01     31.25    104.85   
3                     952.90   435.35   658.25   485.65    117.55     25.00   
4                     345.98    70.40   448.36   169.23     61.49      4.41   
..                       ...      ...      ...      ...       ...       ...   
169                  2027.76    37.45   995.96   356.16    826.90     39.50   
170                  1422.75   203.50   295.00    48.00     26.75      5.50   
171                   282.80     9.00   151.50     8.50      0.00     13.40   
172                   645.70   229.80    34.40    48.50     61.00      0.00   
173                   259.40     9.00    18.25   333.90    626.50   1179.25   

     KR_KH_T  
0       0.00  
1      68.40  
2     

In [282]:
print(Y_T)
print(Y_T.shape)

0      12351.63
1       9285.66
2       3249.62
3       2674.70
4       1099.87
         ...   
169     4283.73
170     2001.50
171      465.20
172     1019.40
173     2426.30
Name: TOTAL_OPUT_T, Length: 170, dtype: float64
(170,)


## **4. Perhitungan Regresi Linear**

In [283]:
X_T = X_T.values
Y_T = Y_T.values.reshape(-1, 1)

#### 4.1. Train Test Split

In [284]:
X_train_T, X_test_T, Y_train_T, Y_test_T = train_test_split(X_T, Y_T, test_size=0.2, random_state=42)

In [285]:
print(f"Shape data latih sumbu X (Variabel Independen) : {X_train_T.shape}")
print(f"Shape data latih sumbu Y (Variabel dependen) : {Y_train_T.shape}")
print("\n============================================================\n")
print(f"Shape data uji sumbu X (Variabel Independen) : {X_test_T.shape}")
print(f"Shape data uji sumbu Y (Variabel dependen) : {Y_test_T.shape}")

Shape data latih sumbu X (Variabel Independen) : (136, 7)
Shape data latih sumbu Y (Variabel dependen) : (136, 1)


Shape data uji sumbu X (Variabel Independen) : (34, 7)
Shape data uji sumbu Y (Variabel dependen) : (34, 1)


#### 4.2. Model Regresi Linear

In [286]:
# Menambahkan variabel tambahan sebagai intercept/konstanta sesuai pada rumus regresi linear
X_train_T_intercept = np.hstack((np.ones((X_train_T.shape[0], 1)), X_train_T))
X_test_T_intercept = np.hstack((np.ones((X_test_T.shape[0], 1)), X_test_T))

In [287]:
# Cari nllai transpose dari matriks X
X_transpose = X_train_T_intercept.T

print(X_transpose)
print(X_transpose.shape)

[[1.000000e+00 1.000000e+00 1.000000e+00 ... 1.000000e+00 1.000000e+00
  1.000000e+00]
 [2.226400e+03 1.950000e+02 1.161500e+02 ... 5.883773e+03 1.821320e+03
  3.311200e+03]
 [4.239000e+02 6.911500e+02 4.307000e+02 ... 1.609880e+03 2.100000e+01
  4.921000e+02]
 ...
 [3.930500e+03 2.280000e+01 3.125000e+01 ... 7.659260e+03 3.120000e+01
  2.321250e+03]
 [2.190000e+01 1.808000e+02 1.048500e+02 ... 1.627700e+02 5.707000e+01
  4.000000e+00]
 [4.180000e+01 5.000000e+00 5.745000e+01 ... 2.256000e+01 0.000000e+00
  0.000000e+00]]
(8, 136)


In [288]:
# Kalikan Matriks Transpose X dengan Matriks X itu sendiri
multiplication_X = X_transpose @ X_train_T_intercept

print(multiplication_X)
print(multiplication_X.shape)

[[1.36000000e+02 3.81851117e+05 1.64749030e+05 3.35905157e+05
  1.76127699e+05 1.85221015e+05 2.50339553e+04 5.70090000e+03]
 [3.81851117e+05 3.02616564e+09 1.71414664e+09 2.49368513e+09
  1.35780350e+09 1.58329259e+09 7.21896618e+07 4.25291966e+07]
 [1.64749030e+05 1.71414664e+09 1.66508550e+09 1.36139931e+09
  9.74053297e+08 1.06832544e+09 4.66752504e+07 4.56399836e+07]
 [3.35905157e+05 2.49368513e+09 1.36139931e+09 2.50939074e+09
  1.19401182e+09 1.33840096e+09 5.65303887e+07 3.88243589e+07]
 [1.76127699e+05 1.35780350e+09 9.74053297e+08 1.19401182e+09
  8.75737021e+08 8.76833214e+08 3.75212476e+07 2.55102020e+07]
 [1.85221015e+05 1.58329259e+09 1.06832544e+09 1.33840096e+09
  8.76833214e+08 1.13713625e+09 4.27233123e+07 3.32626231e+07]
 [2.50339553e+04 7.21896618e+07 4.66752504e+07 5.65303887e+07
  3.75212476e+07 4.27233123e+07 1.03652660e+08 1.44974914e+06]
 [5.70090000e+03 4.25291966e+07 4.56399836e+07 3.88243589e+07
  2.55102020e+07 3.32626231e+07 1.44974914e+06 6.82661048e+06]]

In [289]:
# Cari invers dari perkalian matriks sebelumnya
inversed_multiplication_X = np.linalg.inv(multiplication_X)

print(inversed_multiplication_X)
print(inversed_multiplication_X.shape)

[[ 1.28898623e-02 -1.87026746e-06  1.37026982e-06 -2.14244235e-07
  -1.68211625e-06  9.94610356e-07 -2.03932133e-06 -5.18262468e-06]
 [-1.87026746e-06  3.13731301e-09 -1.00364007e-09 -1.83853521e-09
   4.08294002e-10 -1.40632265e-09  9.04022957e-11  4.49009513e-09]
 [ 1.37026982e-06 -1.00364007e-09  2.32504758e-09  5.44834393e-10
  -1.73221954e-09 -1.19611411e-10 -2.09734076e-10 -6.43420678e-09]
 [-2.14244235e-07 -1.83853521e-09  5.44834393e-10  2.47012384e-09
  -1.12235369e-09  1.06465884e-10  1.35825015e-10 -2.41132474e-09]
 [-1.68211625e-06  4.08294002e-10 -1.73221954e-09 -1.12235369e-09
   7.22598847e-09 -3.05922098e-09  9.33874954e-11  4.70866110e-09]
 [ 9.94610356e-07 -1.40632265e-09 -1.19611411e-10  1.06465884e-10
  -3.05922098e-09  5.19702917e-09 -2.19676432e-10 -5.71906221e-09]
 [-2.03932133e-06  9.04022957e-11 -2.09734076e-10  1.35825015e-10
   9.33874954e-11 -2.19676432e-10  1.01495917e-08  3.35523636e-10]
 [-5.18262468e-06  4.49009513e-09 -6.43420678e-09 -2.41132474e-09
   

In [290]:
# Kalikan matriks transpose X dengan matriks output Y_T
multiplication_Y = X_transpose @ Y_train_T

print(multiplication_Y)
print(multiplication_Y.shape)

[[1.27458887e+06]
 [1.02898124e+10]
 [6.87532541e+09]
 [8.99224270e+09]
 [5.34147030e+09]
 [6.07997439e+09]
 [3.60742270e+08]
 [1.94042724e+08]]
(8, 1)


In [291]:
# Kalikan semua nilai tadi untuk memperoleh nilai koefisien untuk masing - masing variabel independen serta konstanta
coefficient = inversed_multiplication_X @ multiplication_Y

In [292]:
print(coefficient)
print(coefficient.shape)

[[-2.72848411e-11]
 [ 1.00000000e+00]
 [ 1.00000000e+00]
 [ 1.00000000e+00]
 [ 1.00000000e+00]
 [ 1.00000000e+00]
 [ 1.00000000e+00]
 [ 1.00000000e+00]]
(8, 1)


In [293]:
Y_pred = X_test_T_intercept @ coefficient

#### 4.3. Evaluasi Model

In [294]:
mse = mean_squared_error(Y_test_T, Y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(Y_test_T, Y_pred)

In [295]:
print("\nEvaluasi Model:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-Squared (R²): {r2}")

# Menampilkan hasil prediksi dan nilai aktual
print("\nPerbandingan Y aktual vs Y prediksi:")
for actual, pred in zip(Y_test_T, Y_pred):
    print(f"Y aktual: {actual[0]}, Y prediksi: {pred[0]}")


Evaluasi Model:
Mean Squared Error (MSE): 6.7097433933835075e-22
Root Mean Squared Error (RMSE): 2.590317237981384e-11
R-Squared (R²): 1.0

Perbandingan Y aktual vs Y prediksi:
Y aktual: 2599.311, Y prediksi: 2599.310999999978
Y aktual: 1685.6, Y prediksi: 1685.5999999999806
Y aktual: 7750.799999999999, Y prediksi: 7750.80000000002
Y aktual: 9727.28, Y prediksi: 9727.28000000002
Y aktual: 363.45, Y prediksi: 363.44999999997276
Y aktual: 16403.0, Y prediksi: 16402.99999999998
Y aktual: 2001.5, Y prediksi: 2001.4999999999795
Y aktual: 3290.63, Y prediksi: 3290.629999999992
Y aktual: 3686.2499999999973, Y prediksi: 3686.2499999999663
Y aktual: 10700.970000000001, Y prediksi: 10700.969999999985
Y aktual: 11845.215000000002, Y prediksi: 11845.214999999966
Y aktual: 5237.34, Y prediksi: 5237.339999999991
Y aktual: 1907.77, Y prediksi: 1907.7699999999786
Y aktual: 49154.0, Y prediksi: 49154.00000000003
Y aktual: 9350.89, Y prediksi: 9350.890000000018
Y aktual: 6952.469999999997, Y prediksi: 

# **GAUSS ELIMINATION**

Karena metode sebelumnya masih bermasalah, dicoba juga mencari koefisien dengan manggunakan metode eliminasi gauss

In [296]:
# Function to perform Gaussian Elimination
def gauss_elimination(A, b):
    n = len(b)
    # Forward elimination
    for i in range(n):
        # Partial pivoting
        max_row = max(range(i, n), key=lambda x: abs(A[x][i]))
        A[[i, max_row]] = A[[max_row, i]]
        b[[i, max_row]] = b[[max_row, i]]
        
        # Make upper triangular
        for j in range(i + 1, n):
            factor = A[j][i] / A[i][i]
            A[j, i:] -= factor * A[i, i:]
            b[j] -= factor * b[i]

    # Back substitution
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i][i]
    return x

In [297]:

# Function to perform regression using Gaussian Elimination
def linear_regression_gauss(filtered_data):
    # Define dependent and independent variables
    Y_T = filtered_data["TOTAL_OPUT_T"]
    X_T = [
        col
        for col in filtered_data.columns
        if '_T' in col and col != "TOTAL_OPUT_T"
    ]
    X_T = filtered_data[X_T].values
    Y_T = Y_T.values

    # Add intercept to X_T
    X_T_intercept = np.hstack((np.ones((X_T.shape[0], 1)), X_T))
    
    # Formulate normal equation: A @ beta = b
    A = X_T_intercept.T @ X_T_intercept
    b = X_T_intercept.T @ Y_T

    # Solve using Gaussian elimination
    coefficients = gauss_elimination(A.copy(), b.copy())
    return coefficients

In [298]:
gauss_coeff = linear_regression_gauss(data_T)
print(gauss_coeff)

[-3.89095804e-12  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00]


In [299]:
Y_pred_gauss = X_test_T_intercept @ gauss_coeff

In [300]:
mse = mean_squared_error(Y_test_T, Y_pred_gauss)
rmse = np.sqrt(mse)
r2 = r2_score(Y_test_T, Y_pred)

In [301]:
print("\nEvaluasi Model:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-Squared (R²): {r2}")

# Menampilkan hasil prediksi dan nilai aktual
print("\nPerbandingan Y aktual vs Y prediksi:")
for actual, pred in zip(Y_test_T, Y_pred_gauss):
    print(f"Y aktual: {actual[0]}, Y prediksi: {pred[0]}")


Evaluasi Model:
Mean Squared Error (MSE): 1.9916795965752023e-22
Root Mean Squared Error (RMSE): 1.4112687896269804e-11
R-Squared (R²): 1.0

Perbandingan Y aktual vs Y prediksi:


IndexError: invalid index to scalar variable.

# **LIBRARY**


In [118]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score, KFold

In [119]:
X_train_T, X_test_T, y_train_t, y_test_t = train_test_split(X_T, Y_T, test_size=0.2, random_state=42)

In [120]:
# Inisialisasi Model & Latih Model 
model_t = LinearRegression()
model_t.fit(X_train_T, y_train_t)

LinearRegression()

In [121]:
# Nilai intercept untuk model TOTAL_T
intercept_t = model_t.intercept_
coefficients_t = model_t.coef_

print("Intercept untuk TOTAL_T:", intercept_t)
print("Koefisien untuk TOTAL_T:", coefficients_t)


Intercept untuk TOTAL_T: [-1.16415322e-10]
Koefisien untuk TOTAL_T: [[1. 1. 1. 1. 1. 1. 1.]]


In [217]:
y_pred_t = model_t.predict(X_test_T)

# Evaluate for Total_T
print("== Model Performance for TOTAL_OPUT_T ==")
print(f"MSE: {mean_squared_error(y_test_t, y_pred_t):.2f}, \nR2: {r2_score(y_test_t, y_pred_t):.2f}")



== Model Performance for TOTAL_OPUT_T ==
MSE: 0.00, 
R2: 1.00


In [123]:
# Function to preprocess the data from the selected sheet
def preprocess_data(dataset, sheet_name):
    # Load the data for the selected plant type
    data = pd.read_excel(dataset, sheet_name=sheet_name, header=[0, 1])
    data.columns = [" ".join(col).strip() for col in data.columns.values]
    data.columns = (
        data.columns.str.strip()
        .str.replace(" ", "_")
        .str.replace("/", "_")
        .str.replace("-", "_")
    )
    data.rename(
        columns={
            "TAHUN_Unnamed:_0_level_1": "TAHUN",
            "NO_PROV_Unnamed:_1_level_1": "KODE_PROVINSI",
            "PROV_Unnamed:_2_level_1": "PROVINSI",
        },
        inplace=True,
    )
    data.fillna(0, inplace=True)
    return data